In [3]:
import nltk
import sys
import time
import numpy as np
import pandas as pd
import sklearn.feature_extraction.text as text
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from IPython.display import display

ERROR: You must give at least one requirement to install (see "pip help install")


ModuleNotFoundError: No module named 'sklearn'

In [63]:
data = pd.read_csv('../data/train.csv')
sample = data.sample(frac=0.5)
sample.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
15363,28968ac8536bab0f,Thank you for experimenting with Wikipedia. Yo...,0,0,0,0,0,0
69886,baf589607a8f08dd,Contested deletion \n\nThis page should not be...,0,0,0,0,0,0
61803,a5620459a2c8023c,"Man, that's harsh. And inaccurate.",0,0,0,0,0,0
142291,f911cebeafe71f03,Re: Dougweller \n\nIs Dougweller: 1) A Right b...,1,0,0,0,0,0
6743,11f8692b433a918f,"Ah, sorry, my mistake. I haven't been on wiki ...",0,0,0,0,0,0


In [3]:
sample.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,79786.000000,79786.000000,79786.000000,79786.000000,79786.000000,79786.000000
mean,0.095907,0.009851,0.053017,0.002996,0.049532,0.008397
std,0.294465,0.098764,0.224068,0.054650,0.216978,0.091253
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
import re, string
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [57]:
tf = text.TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )
X = tf.fit_transform(sample['comment_text'])

In [58]:
target_labels = []
y = MultiLabelBinarizer().fit_transform(targets) # TODO: score for other columns too

In [61]:
y

array([[1, 1],
       [1, 1],
       [1, 1]])

In [6]:
(X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size=.2)

In [21]:
len(y_train), sum(y_train)

(63828, 6193)

# Naive Bayes

In [8]:
%%time
bnb = GridSearchCV(
    BernoulliNB(),
    param_grid={'alpha': np.logspace(-2., 2., 50)})
bnb.fit(X_train, y_train)

CPU times: user 4min 1s, sys: 15.3 s, total: 4min 16s
Wall time: 43.3 s


In [9]:
confusion_matrix(bnb.predict(X_test), y_test)

array([[11677,   212],
       [ 2822,  1247]])

In [10]:
f1_score(bnb.predict(X_test), y_test)

0.45115774240231543

# Multinomial Naive Bayes

In [11]:
%%time
mnb = GridSearchCV(
    MultinomialNB(),
    param_grid={'alpha': np.logspace(-2., 2., 50)})
mnb.fit(X_train, y_train)

CPU times: user 2min 51s, sys: 7.02 s, total: 2min 58s
Wall time: 33.2 s


In [12]:
confusion_matrix(mnb.predict(X_test), y_test)

array([[14275,   497],
       [  224,   962]])

In [13]:
f1_score(mnb.predict(X_test), y_test)

0.7274102079395084

# SVM Linear

In [37]:
%%time
import random
random.seed(6)
svm_svc = svm.LinearSVC()
svm_svc.fit(X_train, y_train)

CPU times: user 660 ms, sys: 24.2 ms, total: 684 ms
Wall time: 685 ms


In [38]:
confusion_matrix(svm_svc.predict(X_test), y_test)

array([[14415,   514],
       [   84,   945]])

In [39]:
f1_score(svm_svc.predict(X_test), y_test)

0.7596463022508039

# Logistic Regression

In [17]:
%%time
log_reg = GridSearchCV(
    LogisticRegression(),
    param_grid={'C': [5, 10, 15, 20, 30]})
log_reg.fit(X_train, y_train)

CPU times: user 3min 14s, sys: 3.65 s, total: 3min 17s
Wall time: 34.1 s


In [18]:
log_reg.best_params_

{'C': 30}

In [19]:
confusion_matrix(log_reg.predict(X_test), y_test)

array([[14406,   513],
       [   93,   946]])

In [20]:
f1_score(log_reg.predict(X_test), y_test)

0.7574059247397918

# Naive Bayes SVM

In [22]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_is_fitted
from sklearn.linear_model import LogisticRegression
from scipy import sparse
class NbSvmClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, C=1.0, dual=False, n_jobs=1):
        self.C = C
        self.dual = dual
        self.n_jobs = n_jobs

    def predict(self, x):
        # Verify that model has been fit
        check_is_fitted(self, ['_r', '_clf'])
        return self._clf.predict(x.multiply(self._r))

    def predict_proba(self, x):
        # Verify that model has been fit
        check_is_fitted(self, ['_r', '_clf'])
        return self._clf.predict_proba(x.multiply(self._r))

    def fit(self, x, y):
        # Check that X and y have correct shape
        y = y.values
        x, y = check_X_y(x, y, accept_sparse=True)

        def pr(x, y_i, y):
            p = x[y==y_i].sum(0)
            return (p+1) / ((y==y_i).sum()+1)

        self._r = sparse.csr_matrix(np.log(pr(x,1,y) / pr(x,0,y)))
        x_nb = x.multiply(self._r)
        self._clf = LogisticRegression(C=self.C, dual=self.dual, n_jobs=self.n_jobs).fit(x_nb, y)
        return self

In [40]:
%%time
nbsvm = NbSvmClassifier(C=4, dual=True, n_jobs=-1)
nbsvm.fit(X_train, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


CPU times: user 4.24 s, sys: 105 ms, total: 4.34 s
Wall time: 2.67 s


In [46]:
confusion_matrix(nbsvm.predict(X_test), y_test)

array([[14417,   485],
       [   82,   974]])

In [42]:
f1_score(nbsvm.predict(X_test), y_test)

0.7745526838966204

In [53]:
%%time
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200, n_jobs=-1)
rf.fit(X_train, y_train)

CPU times: user 3h 1min 13s, sys: 1min 4s, total: 3h 2min 18s
Wall time: 17min 1s


In [54]:
f1_score(rf.predict(X_test), y_test)

0.5598436736687835

In [64]:
!pip install keras

    100% |████████████████████████████████| 317kB 2.8MB/s ta 0:00:011
    100% |████████████████████████████████| 51kB 9.5MB/s eta 0:00:01
twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [65]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow'

In [69]:
!pip install tensorflow

  Could not find a version that satisfies the requirement tensorflow (from versions: )
No matching distribution found for tensorflow


In [68]:
!pip install --upgrade pip

  Using cached https://files.pythonhosted.org/packages/c2/d7/90f34cb0d83a6c5631cf71dfe64cc1054598c843a92b400e55675cc2ac37/pip-18.1-py2.py3-none-any.whl
twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
  Found existing installation: pip 10.0.1
    Uninstalling pip-10.0.1:
      Successfully uninstalled pip-10.0.1


In [70]:
python

NameError: name 'python' is not defined

In [71]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequence 
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding

data = ('../data/train.csv')


Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow'